<a href="https://colab.research.google.com/github/anshupandey/Generative-AI-for-Professionals/blob/main/EY2024/Additional/Prompt_Engineering_Techniques.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Prompt Engineering Techniques

---




### Install Vertex AI SDK for Python


In [ ]:
! pip3 install --user --quiet openai

In [ ]:
api_key = "xxxxxxxxxxxxxxxxxxxxxxx"
api_version = "2023-07-01-preview" # "2023-05-15", 2024-02-01
azure_endpoint = "https://xxxxxxxxx.openai.azure.com/"
deployment_name = "xxxxxxxxxxxx"

In [ ]:
from openai import AzureOpenAI

# gets the API Key from environment variable AZURE_OPENAI_API_KEY
client = AzureOpenAI(
    # https://learn.microsoft.com/en-us/azure/ai-services/openai/reference#rest-api-versioning
    api_version=api_version,
    # https://learn.microsoft.com/en-us/azure/cognitive-services/openai/how-to/create-resource?pivots=web-portal#create-a-resource
    azure_endpoint=azure_endpoint,
    api_key = api_key,

)

In [ ]:
def generate_response(prompt,temp=0.0):
  response = client.chat.completions.create(
      messages=[{"role":"system",'content':"You are an expert programmer, you follow standard best practices for answering coding questions."},
            {"role":"user",'content':prompt}],
      model = deployment_name,
      temperature=temp,
  )
  return response.choices[0].message.content

We will use Gemini 1.5 Flash model for these given below prompting techniques

### There are different prompting techniques:
**1) Multi-turn conversation or Chatbot with conversation history** <br>
**2) Zero-shot Prompting** <br>
**3) Few-shot Prompting** <br>
**4) Chain-of-Thought Prompting** <br>
**5) Tree-of-Thought Prompting** <br>
**6) ReAct Prompting(Reason and Act)** <br>

**Multi-turn Conversation**

In [ ]:
prompt = ""

while True:
  user = input("You: ")
  if user == "quit":
    break
  prompt = prompt + "User: " + user + "Assistant : "
  ans=generate_response(prompt)
  print("Assistant: ", ans)
  prompt = prompt + ans + "\n"


You: hi
Assistant:  Hi! 👋  How can I help you today? 😊 

You: how are you?
Assistant:  I'm doing well, thank you for asking!  How can I help you today? 😊 

You: I want to learn Generative AI
Assistant:  That's great! Generative AI is a fascinating field. What specifically interests you about it?  

For example, do you want to:
* **Understand the basics?** 
* **Learn about specific generative AI models like DALL-E or ChatGPT?**
* **Explore its applications in different industries?**

Let me know, and I can provide you with information or resources to get you started! 

You: Who is Elon Musk?
Assistant:  Elon Musk is a business magnate, entrepreneur, and investor. He is the founder, CEO, and Chief Engineer of SpaceX; early investor, CEO, and Product Architect of Tesla, Inc.; founder of The Boring Company; and co-founder of Neuralink and OpenAI. 

You: What was the topic, that I wanted to learn?
Assistant:  You wanted to learn about **Generative AI**. 

You: quit


**Zero-shot prompting**

Zero-shot prompting means that the prompt used to interact with the model won't contain examples or demonstrations. The zero-shot prompt directly instructs the model to perform a task without any additional examples to steer it.

In [ ]:
prompt='''
Message: Hi alex, thanks for the thoughtful birthday card!
Sentiment: ?
'''

ans=generate_response(prompt)

print(ans)


Appreciation 



**Few-shot prompting**

Few-shot prompting can be used as a technique to enable in-context learning where we provide demonstrations in the prompt to steer the model to better performance. The demonstrations serve as conditioning for subsequent examples where we would like the model to generate a response.

In [ ]:
prompt = """
Message: Hi Dad, you're 20 minutes late to my piano recital!
Sentiment: Negative

Message: Can't wait to order pizza for dinner tonight
Sentiment: Positive

Message: Hi Amit, thanks for the thoughtful birthday card!
Sentiment: ?
"""

ans=generate_response(prompt)

print(ans)

Sentiment: Positive 



### Specifying the Output Format
- You can also specify the format in which you want the model to respond.
- In the example below, you are asking to "give a one word response".

In [ ]:
prompt = """
Message: Hi Dad, you're 20 minutes late to my piano recital!
Sentiment: Negative

Message: Can't wait to order pizza for dinner tonight
Sentiment: Positive

Message: Hi Amit, thanks for the thoughtful birthday card!
Sentiment: ?

Give a one word response.
"""

ans=generate_response(prompt)

print(ans)

Positive 



**Chain-of-Thought Prompting**
chain-of-thought (CoT) prompting enables complex reasoning capabilities through intermediate reasoning steps. You can combine it with few-shot prompting to get better results on more complex tasks that require reasoning before responding.

In [ ]:
prompt = """
The odd numbers in this group add up to an even number: 4, 8, 9, 15, 12, 2, 1.
A: Adding all the odd numbers (9, 15, 1) gives 25. The answer is False.

The odd numbers in this group add up to an even number: 17,  10, 19, 4, 8, 12, 24.
A: Adding all the odd numbers (17, 19) gives 36. The answer is True.

The odd numbers in this group add up to an even number: 16,  11, 14, 4, 8, 13, 24.
A: Adding all the odd numbers (11, 13) gives 24. The answer is True.

The odd numbers in this group add up to an even number: 17,  9, 10, 12, 13, 4, 2.
A: Adding all the odd numbers (17, 9, 13) gives 39. The answer is False.

The odd numbers in this group add up to an even number: 15, 32, 5, 13, 82, 7, 1.
A:
"""

ans=generate_response(prompt)

print(ans)

Adding all the odd numbers (15, 5, 13, 7, 1) gives 41. The answer is False. 



**Tree of Thoughts (ToT)**

Tree of Thoughts (ToT) prompting is a framework that generalizes over chain-of-thought prompting and encourages exploration over thoughts that serve as intermediate steps for general problem-solving with language models.


How does it work?

- ToT prompting breaks problems down into smaller parts, similar to CoT prompting, but goes further by combining this with the ability to explore multiple solution paths in parallel, forming a tree instead of a single chain. Each thought is generated or solved independently and passed to the next step, allowing the model to self-evaluate and decide whether to continue with that path or choose another.

In [ ]:

prompt = """
Imagine 5 different experts are answering this question.
They will brainstorm the answer step by step reasoning carefully and taking all facts into consideration
All experts will write down 1 step of their thinking,
then share it with the group.
They will each critique their response, and the all the responses of others
They will check their answer based on science and the laws of physics
They will keep going through steps until they reach their conclusion taking into account the thoughts of the other experts
If at any time they realise that there is a flaw in their logic they will backtrack to where that flaw occurred
If any expert realises they're wrong at any point then they acknowledges this and start another train of thought
Each expert will assign a likelihood of their current assertion being correct
Continue until the experts agree on the single most likely location
The question is.
1. Carlos is at the swimming pool.
2. He walks to the locker room, carrying a towel.
3. He puts his watch in the towel and carries the towel tightly to a lounger at the poolside.
4. At the lounger he opens and vigorously shakes the towel, then walks to the snack bar.
5. He leaves the towel at the snack bar, then walks to the diving board.
6. Later Carlos realises he has has lost his watch. Where is the single most likely location of the watch?
"""


ans=generate_response(prompt)

print(ans)

Here's a breakdown of how the experts might reason through this:

**Expert 1:**  The watch was last seen in the towel.  The towel was left at the snack bar.  Therefore, the watch is most likely at the snack bar. (Likelihood: 80%)

**Expert 2:**  We need to consider all possibilities. The watch could have fallen out of the towel while Carlos was shaking it.  This means the watch could be near the lounger. (Likelihood: 20%)

**Expert 3:**  While it's possible the watch fell out during the shaking, the towel was tightly wrapped around the watch, making that unlikely. The watch was also likely secured within the towel. (Likelihood: 90%) 

**Expert 4:**  Expert 3 is right, the watch was secure.  It's much more likely that the watch is at the snack bar with the towel. (Likelihood: 95%)

**Expert 5:**  We need to consider the possibility of theft.  While unlikely, someone could have taken the watch from the towel at the snack bar. (Likelihood: 5%)

**Group Discussion:**

* The experts agree t

**ReAct Prompting**

ReAct is a framework where LLMs are used to generate both reasoning traces and task-specific actions in an interleaved manner.

Generating reasoning traces allow the model to induce, track, and update action plans, and even handle exceptions. The action step allows to interface with and gather information from external sources such as knowledge bases or environments.

In [ ]:
prompt = """
Solve a question answering task with interleaving
Thought,
Action,
Observation steps.

Thought can reason about the current situation, and Action can be three types:

Search[entity], which searches the exact entity on Wikipedia and returns the first paragraph if it exists. If not, it will return some similar entities to search.

Calculator[query], it performs mathematical calculations based on input query.

Lookup[keyword], which returns the next sentence containing keyword in the current passage.

Finish[answer], which returns the answer and finishes the task.

Question:

Who is Elon Musk?
"""


ans=generate_response(prompt)

print(ans)

Thought: I need to find information about Elon Musk.
Action: Search[Elon Musk]
Observation: Elon Reeve Musk (born June 28, 1971) is a business magnate and entrepreneur. He is the founder, CEO and Chief Engineer of SpaceX; early investor, CEO and Product Architect of Tesla, Inc.; founder of The Boring Company; and co-founder of Neuralink and OpenAI. He is the world's second-wealthiest person, according to the Bloomberg Billionaires Index and Forbes's real-time billionaire list, as of December 2022.
Thought: I have the information about Elon Musk.
Action: Finish[Elon Musk is a business magnate and entrepreneur. He is the founder, CEO and Chief Engineer of SpaceX; early investor, CEO and Product Architect of Tesla, Inc.; founder of The Boring Company; and co-founder of Neuralink and OpenAI.] 



**2. Chain-of-Verification Prompting**

The Chain-of-Verification (CoVe) prompt engineering method aims to reduce hallucinations through a verification loop.

In [ ]:
Question="Name some athletes who were born in United states"

prompt=f'''Here is the question: {Question}.

First, generate a response.

Then, create and answer verification questions based on this response to check for accuracy. Think it through and make sure you are extremely accurate based on the question asked.

After answering each verification question, consider these answers and revise the initial response to formulate a final, verified answer. Ensure the final response reflects the accuracy and findings from the verification process.'''



ans=generate_response(prompt)

print(ans)


## Initial Response:

Some athletes born in the United States include Michael Jordan, Serena Williams, Tom Brady, and Lebron James.

## Verification Questions:

1. **Are all the athletes listed born in the United States?** 
2. **Are all the athletes listed considered "athletes" in the traditional sense?** 
3. **Are there any other famous athletes born in the United States that should be included in the list?**

## Answering Verification Questions:

1. **Yes.**  All the athletes listed were born in the United States.
2. **Yes.**  All listed individuals are considered athletes in the traditional sense.
3. **Yes.**  There are many other famous athletes born in the United States, including  Muhammad Ali, Tiger Woods, and Simone Biles.

## Final, Verified Response:

Some athletes born in the United States include Michael Jordan, Serena Williams, Tom Brady, Lebron James, Muhammad Ali, Tiger Woods, and Simone Biles. 



**3. Step-Back Prompting**

A crucial rule to remember when prompting LLMs is to give them space to 'think'. You don’t want to overly constrain the model such that it can’t explore various solutions.

Chain of thought reasoning is one way to push the model to think through the problem. A simple way to implement this type of reasoning is to add a statement like “think through this task step-by-step” at the end of your prompt.

In [ ]:
Question="What is Quantum Physics?"
prompt=f'''Here is a question or task: {Question}

Let's think step-by-step to answer this:

Step 1) Abstract the key concepts and principles relevant to this question:

Step 2) Use the abstractions to reason through the question:

Final Answer: '''


ans=generate_response(prompt)

print(ans)



## Step 1: Key Concepts and Principles

* **Quantum Mechanics:**  A fundamental theory in physics that describes the physical properties of nature at the scale of atoms and subatomic particles.
* **Quantization:** The idea that certain physical quantities, like energy, can only exist in discrete values (quanta) rather than continuously.
* **Wave-Particle Duality:**  The concept that particles, like electrons, exhibit both wave-like and particle-like properties.
* **Uncertainty Principle:**  States that it is impossible to know both the position and momentum of a particle with absolute certainty.
* **Superposition:**  A state where a quantum system can exist in multiple states simultaneously until measured.
* **Entanglement:**  A phenomenon where two or more quantum particles become interconnected, even when separated by vast distances.

## Step 2: Reasoning

* Quantum physics studies the behavior of matter and energy at the atomic and subatomic levels.
* Unlike classical physics, which

## Thank You